### Interactive Map

#### **Important Note:**

In our project, we used Leadfet.js (part of the JavaScript library) to build our interactive map. We chose Leaflet because it offered more customizability (e.g., dynamic interactions like zooming, panning, and marker placement) and complimented our programming capabilities nicely. 

Due to the fact that our term deliverable must be built within Jupyter Notebook, we replicated our Leaflet map using Folium. Folium is a Python wrapper for Leaflet. It offers the same capabilities but is designed to be compatible in Jupyter environments. If you are interested in comparing the differences, please clone our GitHub project. 

In [43]:
#!pip install ipywidgets ipyleaflet requests
#!pip install folium ipywidgets requests

In [15]:
import requests
import folium
import pandas as pd
from ipywidgets import Dropdown, Button, VBox, HBox, Output
from ipyleaflet import Map, Marker, TileLayer, MarkerCluster
from IPython.display import display
API_BASE = "http://apan-api:3100/api/v1/business" 

GET_STATES_URL = f"{API_BASE}/getStateList"
GET_CITIES_URL = f"{API_BASE}/getCityList"
GET_BUSINESSES_URL = f"{API_BASE}/getBusinessListInCity"
# Fetch states from the API
response = requests.get(GET_STATES_URL)
state_list = response.json().get('result', [])

state_dropdown = Dropdown(
    options=state_list,
    description='State:',
    layout={'width': '300px'}
)
city_dropdown = Dropdown(
    options=[],
    description='City:',
    layout={'width': '300px'}
)

def update_city_options(change):
    state = change['new']
    if state:
        url = f"{GET_CITIES_URL}/{state}"
        resp = requests.get(url)
        city_list = resp.json().get('result', [])
        city_dropdown.options = city_list

state_dropdown.observe(update_city_options, names='value')

map_output = Output()

def display_restaurants(state, city):
    payload = {"state": state, "city": city}
    resp = requests.post(GET_BUSINESSES_URL, json=payload)
    businesses = resp.json().get("result", [])
    
    map_output.clear_output()
    
    if not businesses:
        with map_output:
            print("No restaurants found for this location.")
        return

    # Create map centered on the first restaurant
    m = folium.Map(location=[businesses[0]['latitude'], businesses[0]['longitude']], zoom_start=13)

    for biz in businesses:
        folium.Marker(
            location=[biz['latitude'], biz['longitude']],
            popup=f"<b>{biz['name']}</b><br>{biz['address']}",
            tooltip=biz['name']
        ).add_to(m)

    with map_output:
        display(m)

show_map_button = Button(description="Show Restaurants", button_style='info')

def on_button_click(b):
    if state_dropdown.value and city_dropdown.value:
        display_restaurants(state_dropdown.value, city_dropdown.value)
    else:
        with map_output:
            clear_output()
            print("Please select both a state and a city.")

show_map_button.on_click(on_button_click)

ui = VBox([state_dropdown, city_dropdown, show_map_button, map_output])
display(ui)